In [1]:
from __future__ import division

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from matplotlib import finance

%matplotlib inline

# Loading and Exploring the Data

In [2]:
data = pd.read_csv('WIKI_20160608.csv', names=['Ticker', 'Date', 'Open', 'High', 'Low', 'Close',
                                              'Volume', '7', '8', 'Adj-O', 'Adj-H', 'Adj-L', 'Adj-C',
                                              'Adj-Vol'])

In [3]:
data.head(5)

,Ticker,Date,Open,High,Low,Close,Volume,7,8,Adj-O,Adj-H,Adj-L,Adj-C,Adj-Vol
0,A,1999-11-18,45.50,50.00,40.00,44.00,44739900.0,0.0,1.0,43.584507,47.895063,38.316050,42.147655,44739900.0
1,A,1999-11-19,42.94,43.00,39.81,40.38,10897100.0,0.0,1.0,41.132280,41.189754,38.134049,38.680053,10897100.0
2,A,1999-11-22,41.31,44.00,40.06,44.00,4705200.0,0.0,1.0,39.570901,42.147655,38.373524,42.147655,4705200.0
3,A,1999-11-23,42.50,43.63,40.25,40.25,4274400.0,0.0,1.0,40.710803,41.793232,38.555525,38.555525,4274400.0
4,A,1999-11-24,40.13,41.94,40.00,41.06,3464400.0,0.0,1.0,38.440577,40.174379,38.316050,39.331425,3464400.0


Create a new dataframe with only the adjusted data, data, & ticker symbol.

In [4]:
dat = data[['Ticker', 'Date', 'Adj-O', 'Adj-H', 'Adj-L', 'Adj-C', 'Adj-Vol']]

In [5]:
dat.head(5)

,Ticker,Date,Adj-O,Adj-H,Adj-L,Adj-C,Adj-Vol
0,A,1999-11-18,43.584507,47.895063,38.316050,42.147655,44739900.0
1,A,1999-11-19,41.132280,41.189754,38.134049,38.680053,10897100.0
2,A,1999-11-22,39.570901,42.147655,38.373524,42.147655,4705200.0
3,A,1999-11-23,40.710803,41.793232,38.555525,38.555525,4274400.0
4,A,1999-11-24,38.440577,40.174379,38.316050,39.331425,3464400.0


In [7]:
dat.shape

(14150093, 7)

Create a dataframe for a specific ticker

In [8]:
agilent = dat[dat['Ticker'] == 'A']

In [9]:
agilent.head()

,Ticker,Date,Adj-O,Adj-H,Adj-L,Adj-C,Adj-Vol
0,A,1999-11-18,43.584507,47.895063,38.316050,42.147655,44739900.0
1,A,1999-11-19,41.132280,41.189754,38.134049,38.680053,10897100.0
2,A,1999-11-22,39.570901,42.147655,38.373524,42.147655,4705200.0
3,A,1999-11-23,40.710803,41.793232,38.555525,38.555525,4274400.0
4,A,1999-11-24,38.440577,40.174379,38.316050,39.331425,3464400.0


Pretty straight forward to get a ticker specific set of data.  Below I will create a function to automate the process.

In [10]:
def tick_gettr(ticker):
    data = dat[dat['Ticker'] == ticker]
    return data

I will also create an array for all tickers in the dataset in case that comes in handy.

In [11]:
tickers = dat['Ticker']
tickers = tickers.unique()
len(tickers)

3179

Now I will get a ticker specific set of data to work with

In [13]:
aapl = tick_gettr('AAPL')

## Basic Calculations

I need the components and then the look back period.

Also note that the calculations need to be made for the data historically, not just for the look back period (e.g. look back of 14 days gives us a value just for those 14 days.)  The look back period is how far back the function will look back to make the calculation.

**EXAMPLE:**

Using a lookback period of 14 days will gives us the 14 day calculation for RSI for the entire history of the data set, not just the last 14 days.  I will be able to look & see what the 14 day RSI was for June 25th, 2000 and for the most recent data point as well.

In [15]:
def rs(ticker, look_back):
    """
    A function to find the relative strength of a stock using a user specified
    look back period.
    """
    
    #Get the difference between consecutive adjusted closes.
    difference = ticker['Adj-C'].diff()
    difference = difference[1:]
    
    #Define & clean up the up sessions & down sessions.
    up, down = difference.copy(), difference.copy()
    
    up[up < 0] = 0
    down[down > 0] = 0
    
    #Get the absolute value for the down sessions.
    down = down.abs()
    
    #Get the look_back mean for both up & down sessions.
    upper = up.rolling(window=look_back, center=False).mean()
    downer = down.rolling(window=look_back, center=False).mean()
        
    #Calculate the relative strength
    rs = upper / downer
    
    return rs

In [16]:
def rsi(ticker, look_back):
    """
    A function that determines the relative strength index of a specific stock.
    """
    
    #Get the relative strength.
    r_s = rs(ticker, look_back)
    #Calculate the RSI.
    rsi = 100.0 - 100.0 / (1.0 + r_s)

    return rsi.round(decimals=2)

The Python code to get the RS calculation was greatly informed from this StackOverflow answer:

http://stackoverflow.com/a/29400434/1094634

Also important to note, that some sources will calculate some element of the RSI differently.  Yahoo Finance & ThinkorSwim for instances calculate it the same (and differently from the above calculation) while Freestockcharts.com (Worden) calculates it the same way.

## Buy/Sell Rule:

I will begin with a simple buy/sell rule.  Buy @ the closing price when RSI < 30 & sell @ the closing price when RSI > 70.  The 30 & 70 thresholds are the traditional demarcation points for 'oversold' (<30) & 'overbought' (>70).  These thresholds are ripe for fine tuning.  Initially, I will be using the 14 day average.  The other average that I want to look at is the 9 day.  As with the oversold/overbought level, the optimal average needs to be tested/tuned as well.

Simply, what I want to know is if the RSI predicts higer prices from the buy point.  The actual price itself is not very important (at least in the initial stage.)  If the sell price is consistently higher than buy price, then I will interpret that as the RSI as having some predictive value (whether or not the RSI strategy is more profitable than simply buying the S&P 500 is a separate question.)


First, I will get Apples (AAPL) RSI using the 14 day average.

In [17]:
aapl_rsi = rsi(aapl, 14)

Now I want to add a new column to the dataset to include the RSI

In [22]:
aapl['RSI'] = aapl_rsi

/Users/amagi/anaconda/envs/mlnd/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [23]:
aapl.head(5)

,Ticker,Date,Adj-O,Adj-H,Adj-L,Adj-C,Adj-Vol,RSI
39644,AAPL,1980-12-12,0.436051,0.437871,0.436051,0.436051,117258400.0,NaN
39645,AAPL,1980-12-15,0.415273,0.415273,0.413301,0.413301,43971200.0,NaN
39646,AAPL,1980-12-16,0.384787,0.384787,0.382967,0.382967,26432000.0,NaN
39647,AAPL,1980-12-17,0.392370,0.394342,0.392370,0.392370,21610400.0,NaN
39648,AAPL,1980-12-18,0.403897,0.405717,0.403897,0.403897,18362400.0,NaN


Next I want to remove the first 14 columsn with NaNs for the RSI

In [28]:
aapl = aapl[14:]

In [29]:
aapl.head()

,Ticker,Date,Adj-O,Adj-H,Adj-L,Adj-C,Adj-Vol,RSI
39658,AAPL,1981-01-05,0.513706,0.513706,0.511886,0.511886,8932000.0,64.50
39659,AAPL,1981-01-06,0.490956,0.490956,0.489136,0.489136,11289600.0,64.50
39660,AAPL,1981-01-07,0.470177,0.470177,0.468357,0.468357,13921600.0,66.95
39661,AAPL,1981-01-08,0.460622,0.460622,0.458802,0.458802,9956800.0,63.18
39662,AAPL,1981-01-09,0.483524,0.485344,0.483524,0.483524,5376000.0,65.01


Now I want to create some more columns that will show the prices for days where the RSI is < 30 & > 70.

First I will create the Sell column with corresponding prices.

In [37]:
aapl['Sell'] = pd.Series(0, np.arange(8935))

/Users/amagi/anaconda/envs/mlnd/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [38]:
aapl.ix[aapl.RSI > 70, 'Sell'] = aapl['Adj-C']

Next I will do the same for a Buy column.

In [40]:
aapl['Buy'] = pd.Series(0, np.arange(8935))

/Users/amagi/anaconda/envs/mlnd/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [41]:
aapl.ix[aapl.RSI < 30, 'Buy'] = aapl['Adj-C']

I will also create several columns that represent holding periods of 1, 3, 5, & 10 days.  I think it will be beneficial to see whether or not arbitrary holding periods show better returns then the RSI > 70 sell point.  These holding periods can be played with and they are picked to represent the shorter term nature of this type of trade.

In [44]:
base = aapl.copy()
base = base['Adj-C']

day_1 = base.shift(-1)
day_3 = base.shift(-3)
day_5 = base.shift(-5)
day_10 = base.shift(-10)

In [45]:
aapl['Day 1'] = day_1
aapl['Day 3'] = day_3
aapl['Day 5'] = day_5
aapl['Day 10'] = day_10

/Users/amagi/anaconda/envs/mlnd/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/amagi/anaconda/envs/mlnd/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/amagi/anaconda/envs/mlnd/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

In [46]:
aapl.head(5)

,Ticker,Date,Adj-O,Adj-H,Adj-L,Adj-C,Adj-Vol,RSI,Sell,Buy,Day 1,Day 3,Day 5,Day 10
39658,AAPL,1981-01-05,0.513706,0.513706,0.511886,0.511886,8932000.0,64.50,NaN,NaN,0.489136,0.458802,0.479581,0.498539
39659,AAPL,1981-01-06,0.490956,0.490956,0.489136,0.489136,11289600.0,64.50,NaN,NaN,0.468357,0.483524,0.462594,0.483524
39660,AAPL,1981-01-07,0.470177,0.470177,0.468357,0.468357,13921600.0,66.95,NaN,NaN,0.458802,0.479581,0.464565,0.492928
39661,AAPL,1981-01-08,0.460622,0.460622,0.458802,0.458802,9956800.0,63.18,NaN,NaN,0.483524,0.462594,0.473969,0.498539
39662,AAPL,1981-01-09,0.483524,0.485344,0.483524,0.483524,5376000.0,65.01,NaN,NaN,0.479581,0.464565,0.470177,0.496719


Now I have a complete dataset to begin working with.

# Analyzing the Data